In [3]:
# If you haven't a local copy of sidh-optimizer, run
#
#!pip install https://github.com/sidh-crypto/sidh-optimizer.git

In [4]:
from sidh_optimizer.formulas import *

In [5]:
sympy.init_printing()

## A quick tour

The `formulas` module contains symbolic representations of elliptic curve formulas. These representations can be visualized as `sympy` expressions, but also know about their computational cost.

We start with the classic Edwards addition formula. Here an Edwards curve is a curve of the form

$$E(x^2 + y^2) = E + Dx^2y^2.$$

Dividing both sides by $E$ gives the usual definition of Edwards curves.

Here is the addition formulas in projective coordinates for this generlized Edwards curve.

In [6]:
X, Y, Z = ed_add(Var('X1'), Var('Y1'), Var('Z1'),
                 Var('X2'), Var('Y2'), Var('Z2'),
                 Var('D'), Var('E'))
X, Y, Z

(E*Z1*Z2*(-D*X1*X2*Y1*Y2 + E*Z1**2*Z2**2)*(-X1*X2 - Y1*Y2 + (X1 + Y1)*(X2 + Y2)),
 E*Z1*Z2*(-X1*X2 + Y1*Y2)*(D*X1*X2*Y1*Y2 + E*Z1**2*Z2**2),
 (-D*X1*X2*Y1*Y2 + E*Z1**2*Z2**2)*(D*X1*X2*Y1*Y2 + E*Z1**2*Z2**2))

The formulas know about their computational cost.

In [7]:
cost(X, Y, Z)

13M + 7A + 1S

Note that the above function shares common subexpressions. If we compute the costs individually, we obtain a higher operation count.

In [8]:
X.cost(), Y.cost(), Z.cost()

(5A + 10M + 1S, 2A + 9M + 1S, 2A + 7M + 1S)

We can have a better look at the formulas by using `sympy`

In [9]:
[f.formula.simplify() for f in (X,Y,Z)]

This might be a little hard to check, let's fix some variables, so that we obtain the classic affine formula

In [10]:
X, Y, Z = ed_add(Var('X1'), Var('Y1'), Const(1),
                 Var('X2'), Var('Y2'), Const(1),
                 Var('d'), Const(1))
[f.formula.simplify() for f in (X,Y,Z)]

We have the same for doubling (the formula does not depend on $D,E$).

In [11]:
X, Y, Z = ed_double(Var('X'), Var('Y'), Var('Z'))
[f.formula.simplify() for f in (X,Y,Z)]

## Isogenies

Let's move to the interesting stuff. The isogeny formula from Section 4.3 of https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/edwardsisogenies.pdf, reads

$$ψ(x,y) = \left(x\prod_{i=1}^s\frac{β_i^2 x^2 - α_i^2y^2}{β_i^2 - d^2α_i^2β_i^4x^2y^2},
y\prod_{i=1}^s\frac{β_i^2 y^2 - α_i^2x^2}{β_i^2 - d^2α_i^2β_i^4x^2y^2}\right)$$

where $(±α_i,β_i)$ are the coordinates of the kernel of $ψ$, and $d=D/E$.

Here is a projective version (both for the point coordinates and the curve invariant) of the formula. We separate the computations that depend upon $x,y$ (the *isogeny evaluation*) from those that are independent (the *next curve*); the function returns

- The invaraint of the image curve;
- Costants independent of $x,y$ needed for the isogeny evaluation;
- The evaluation formula.

In [12]:
curve, consts, isog = edwards(3,
                              Var('D'), Var('E'),
                              Var('α'), Var('β'), Var('γ'),
                              Var('X'), Var('Y'), Var('Z'))

In [13]:
[c.formula for c in curve]

⎡ 3  8   3  8⎤
⎣D ⋅β , E ⋅γ ⎦

In [14]:
[[c.formula for c in cs] for cs in consts]

⎡⎡   2  4     2  4     2  4⎤⎤
⎣⎣E⋅β ⋅γ , E⋅α ⋅γ , D⋅α ⋅β ⎦⎦

In [15]:
[f.formula.simplify() for f in isog]

In the latter formula, $A_0, B_0, D_0$ represent the pre-computed values stored in `consts`.

Let's substitute them and check that the formula is correct.

In [16]:
isog2 = [f.formula.simplify().subs({
    'A0':consts[0][0].formula,
    'B0':consts[0][1].formula,
    'D0':consts[0][2].formula,
}) for f in isog]
isog2

⎡  ⎛   2  2  4      2  2  4⎞    ⎛     2  2  4      2  2  4⎞    ⎛     2  4 ⎛ 2 
⎣X⋅⎝E⋅X ⋅β ⋅γ  - E⋅Y ⋅α ⋅γ ⎠, Y⋅⎝- E⋅X ⋅α ⋅γ  + E⋅Y ⋅β ⋅γ ⎠, Z⋅⎝- D⋅α ⋅β ⋅⎝X  

   2    2⎞      2  2  4⎞⎤
+ Y  - Z ⎠ + E⋅Z ⋅β ⋅γ ⎠⎦

These formulas look correct, but to be even more convinced, let's substitute the curve equation

$$X^2+Y^2-Z^2 = \frac{D X^2 Y^2}{E Z^2},$$

divide all members by $Eγ^6Z^3$, and replace affine coordinates.

In [17]:
X, Y, Z, a, b, c, D, E = sympy.symbols('X Y Z α β γ D E')
[(i.subs({X**2+Y**2-Z**2: D*X**2*Y**2/(E*Z**2)}) / (E*c**6*Z**3)).expand().subs({
    a/c: a,
    b/c: b,
    X/Z: X,
    Y/Z: Y,
    D/E: D,
}).simplify() for i in isog2]

⎡  ⎛ 2  2    2  2⎞    ⎛   2  2    2  2⎞     2  2  2  2  4    2⎤
⎣X⋅⎝X ⋅β  - Y ⋅α ⎠, Y⋅⎝- X ⋅α  + Y ⋅β ⎠, - D ⋅X ⋅Y ⋅α ⋅β  + β ⎦

Let's now move to the costs. The *next curve* computation contains both the curve invariant and the pre-computed constants

In [18]:
cost(*curve, *sum(consts, ()))

9M + 9S

While the *isogeny evaluation* only comprises the latter formulas

In [19]:
cost(*isog)

8A + 8M + 3S

Obviously, these costs must be taken as upper bounds on the cost of isogeny evaluation. There are indeed (much) better formulas known for 3-isogenies of Edwards curves.

Finally, let's find out how the costs increase with the degree.

In [20]:
def get_costs(ell):
    E, c, I = edwards(ell)
    pre, consts, isog = cost(*E, *sum(c, ())), cost(*sum(c,())), cost(*I)
    return consts, pre-consts, isog

prev = [Cost()]*3
for ell in range(3, 20, 2):
    next = get_costs(ell)
    print([n-p for n, p in zip(next, prev)])
    prev = next

[5S + 5M, 4S + 4M, 8A + 8M + 3S]
[9S + 15M + 7A, 2S + 2M + 0A, 6A + 8M + 0S]
[6S + 18M + 7A, 0S + 4M + 0A, 8M + 6A + 0S]
[6S + 18M + 7A, 2S + 0M + 0A, 8M + 6A + 0S]
[6S + 18M + 7A, 0S + 4M + 0A, 8M + 6A + 0S]
[6S + 18M + 7A, 0S + 2M + 0A, 8M + 6A + 0S]
[6S + 18M + 7A, 0S + 4M + 0A, 8M + 6A + 0S]
[6S + 18M + 7A, 2S + -2M + 0A, 8M + 6A + 0S]
[6S + 18M + 7A, 0S + 4M + 0A, 8M + 6A + 0S]


We see that the cost of the *next curve* operation for $ℓ=2s+1$ is $(18s-13)M + (9s-4)S$ plus a (small) variable number of multiplications and squarings coming from the computation of the next invariant (the variability comes from the way $d^ℓ$ is computed).

The cost for the *isogeny evaluation* is $8sM + 3S$.

## Montgomery curves

Using the formulas from https://eprint.iacr.org/2017/504.pdf, https://eprint.iacr.org/2017/1198.pdf.
